In [1]:
import os,sys
import math


os.getcwd()

'/home/fin/tmp/fis-ski'

In [55]:
import unicodecsv

def read_lines(reader):
    headers = reader.next()
    for x in reader:
        yield dict(zip(headers,x))

all_ergebnisse = []
errorcount=0
errors = []

import re
numbersonly = re.compile('=([0-9]+)')

all_races = {}

with open('fin_output_2.csv') as ergebnisse:#, open('subpages.csv') as subpages:
    er = read_lines(unicodecsv.reader(ergebnisse))
     
    #sr = unicodecsv.reader(subpages)
    
    #all_races = dict((numbersonly.findall(x['url'])[0],x,) for x in read_lines(sr))
    #for r in all_races.values():
    #    ds = r['date'].split('.')
    #    r['year'] = ds[2]
    #    r['month'] = ds[1]
    #    r['day'] = ds[0]

    
    for e in er:
        try:
            #e['race'] = all_races[numbersonly.findall(e['source_url'])[0]]
            e['date'] = e['subevent_Date']
            ds = e['date'].split('.')
            e['year'] = int(ds[2],10)
            e['month'] = int(ds[1],10)
            e['day'] = int(ds[0],10)
            all_ergebnisse.append(e)
        except:
            errorcount+=1
            errors.append(e)

In [56]:
def sortgroup(xs, keyfun):
    import itertools
    return itertools.groupby(sorted(xs,key=keyfun),key=keyfun)

by_race = dict([(x,
                 list(sorted(
                    [z for z in y if z['Rank'] and z['Rank'].isdigit()]
                    , key=lambda e: int(e['Rank'],10))),
                )
                for (x,y,) in sortgroup(all_ergebnisse, lambda x: x['source_url'])
               ])

In [57]:
pointreplaced = 0

for raceurl,finishers in by_race.iteritems():

    last = None
    for f in finishers:
        f['rang'] = int(f['Rank'],10)

        if f['Total Time'].strip() and '?' not in f['Total Time']:
            if last:
                if ':' in last['Total Time'] and not ':' in f['Total Time']:
                    pointreplaced+=1
                    f['Total Time'] = f['Total Time'].replace('.',':',1)
            if ':' in f['Total Time']:
                minutes,rest = f['Total Time'].split(':',1)
            else:
                minutes = '0'
                rest = f['Total Time']
            
            try:
                seconds,nths = re.split('[\.,]',rest)
            except ValueError,e:
                seconds = rest
                nths = '0'

            minutes = int(minutes,10)
            seconds = int(seconds,10)
            nths_i = int(nths,10)
            nths_f = nths_i/(pow(10.0,len(nths)))
            finalseconds = minutes*60 + seconds + nths_f
            

            f['mytime'] = finalseconds

            f['diff'] = f['mytime']-finishers[0]['mytime']
            if last and hasattr(last,'mytime'):
                if f['diff']<last['diff']:
                    print 'time travel?'
                    print f['Total Time'],f['mytime'],last['Total Time'],last['mytime'],f['source_url']
                    print seconds
                    print nths, len(nths), nths_i, nths_f
    
        last = f

In [58]:
with open('ergebnisse_w_diffs.csv','w') as f:
    cw = unicodecsv.writer(f)
    f = by_race.values()[0][0]
    fk = [x for x in f.keys() if x!='race' and x!='source_url']
    rk = []#f['race'].keys()
    headers = fk+rk
    cw.writerow(headers)
    for y in by_race.values():
        for x in y:
            if x['Total Time'].strip():
                cw.writerow(
                    [x[k] for k in fk] + [x['race'][k] for k in rk])

In [41]:
import itertools
all_diffs = [x for x in itertools.chain(*by_race.values()) if x.get('diff',None)]

In [42]:
import collections

MAX_SECONDS = 4

all_fitting = {k:[{'Rank': x['Rank'],
                   'Name': x['Name'],
                   'Time': x['Total Time'],
                   'diff': x['diff'],
                   'time': x['mytime'],
                   'race': x,
                  } for x in vv if x.get('diff',None) is not None and x['diff']<=MAX_SECONDS] for k,vv in by_race.iteritems()
               if vv}

In [43]:
#only needed because earlier versions did not have inline race/event data
result = []
for k,v in all_fitting.iteritems():
    if not v:
        continue
    r = {}
    r.update(v[0]['race'])
    r['year'] = v[0]['race']['year']
    r['month'] = v[0]['race']['month']
    r['ranks'] = v
    result.append(r)
#result = all_fitting.values()

In [44]:
v[0]['race']

{u'Bib': u'',
 u'Diff.': u'&nbsp',
 u'FIS Code': u'510993',
 u'FIS Points': u'0.00',
 u'Name': u'ALBRECHT Daniel',
 u'Nation': u'SUI',
 u'Rank': u'1',
 u'Run 1': u'',
 u'Run 2': u'',
 u'Skis': u'?????',
 u'Total Time': u'2:28.99',
 u'Year': u'1983',
 'date': u'08.02.2007',
 'day': 8,
 'diff': 0.0,
 'month': 2,
 'mytime': 148.99,
 'rang': 1,
 u'source_url': u'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=49075',
 u'subevent_': u'',
 u'subevent_Category': u'WSC',
 u'subevent_Codex': u'0389',
 u'subevent_Comments': u'',
 u'subevent_Date': u'08.02.2007',
 u'subevent_Discipline': u'Super Combined',
 u'subevent_Download': u'Results, Analysis, Standings (822.51 kb)Results 2nd run (398.46 kb)Start List 2nd run (391.93 kb)Start List 1st run (401.44 kb)',
 u'subevent_Gender': u'M',
 u'subevent_Nation': u'SWE',
 u'subevent_Place': u'AreRUNCETLOCRun 112:30Run 216:00',
 u'subevent_event_CategoryCat.': u'COM',
 u'subevent_event_Codex': u'',
 u'subevent_event_Comments': u'',
 u'subev

In [45]:
last_season_races = [x for x in result
 if ('2015'==x['year'] and int(x['month'],10)>6)
 or (x['year']=='2016' and int(x['month'],10)<=6)]

In [46]:
x

{u'Bib': u'',
 u'Diff.': u'&nbsp',
 u'FIS Code': u'510993',
 u'FIS Points': u'0.00',
 u'Name': u'ALBRECHT Daniel',
 u'Nation': u'SUI',
 u'Rank': u'1',
 u'Run 1': u'',
 u'Run 2': u'',
 u'Skis': u'?????',
 u'Total Time': u'2:28.99',
 u'Year': u'1983',
 'date': u'08.02.2007',
 'day': 8,
 'diff': 0.0,
 'month': 2,
 'mytime': 148.99,
 'rang': 1,
 'ranks': [{'Name': u'ALBRECHT Daniel',
   'Rank': u'1',
   'Time': u'2:28.99',
   'diff': 0.0,
   'race': {u'Bib': u'',
    u'Diff.': u'&nbsp',
    u'FIS Code': u'510993',
    u'FIS Points': u'0.00',
    u'Name': u'ALBRECHT Daniel',
    u'Nation': u'SUI',
    u'Rank': u'1',
    u'Run 1': u'',
    u'Run 2': u'',
    u'Skis': u'?????',
    u'Total Time': u'2:28.99',
    u'Year': u'1983',
    'date': u'08.02.2007',
    'day': 8,
    'diff': 0.0,
    'month': 2,
    'mytime': 148.99,
    'rang': 1,
    u'source_url': u'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=49075',
    u'subevent_': u'',
    u'subevent_Category': u'WSC',
    u's

In [47]:
for race in last_season_races:
    tmp = [x['diff'] for x in race['ranks'] if int(x['Rank'],10)<=5]
    race['knappheit'] = sum(tmp)/len(tmp)
    race['diffzweiter'] = race['ranks'][1]['diff']
    race['finishersinnsecs'] = len([x for x in race['ranks'] if x['diff']<=MAX_SECONDS])

In [48]:
by_disc = dict((x,list(y),) for x,y in sortgroup(last_season_races,
                                                 lambda x: '%s (%s)' % (x['disziplin'],x['gender'],)))

In [49]:
def mean(numbers):
    return sum(numbers)/(1.0*len(numbers))

MAX_RANK = 15

with open('out.csv','w') as f, open('debug.csv','w') as f2:
    cw = unicodecsv.writer(f)
    cw2 = unicodecsv.writer(f2)
    
    cw.writerow(['Disziplin','Rennen']+
               list(itertools.chain.from_iterable([['Platz %s'%i, 'Name %s'%i] for i in xrange(1,MAX_RANK+1)])))

    for disc,discraces in by_disc.iteritems():
        closest = sorted(discraces, key=lambda x:x['diffzweiter'])[0]
        cw.writerow([disc,'closest (%s / %s)'%(closest['ort'],closest['date'],)]+
                    list(itertools.chain.from_iterable([
                     [round(x['diff'],2),x['Name']]
                      for x in closest['ranks'][:MAX_RANK+1]
                     ]
                   )
                    ))
        all_fitting = [[[x['diff'],x['Name']] for x in reduce(lambda x,y: x+y, [z['ranks'] for z in discraces],[]) if int(x['Rank'],10)==y] for y in xrange(1,MAX_RANK+1)]
        average = [mean([y[0] for y in x]) for x in all_fitting]
        cw.writerow([disc,'mean']+
                    list(itertools.chain.from_iterable([
                        [round(x,2),'%d. Platz' % (i+1)] for i,x in enumerate(average)
                    ]))
           )
        for x in all_fitting:
            diffs = [y[0] for y in x]
            x_sorted = sorted(x, key=lambda y: y[0])
            cw2.writerow([disc]+['%f:%s'%(y[0],y[1],) for y in x_sorted]+['||',mean(diffs),sorted(diffs)[int(math.floor(len(x)/2))]])

In [50]:
x

{u'Bib': u'',
 u'Diff.': u'&nbsp',
 u'FIS Code': u'510993',
 u'FIS Points': u'0.00',
 u'Name': u'ALBRECHT Daniel',
 u'Nation': u'SUI',
 u'Rank': u'1',
 u'Run 1': u'',
 u'Run 2': u'',
 u'Skis': u'?????',
 u'Total Time': u'2:28.99',
 u'Year': u'1983',
 'date': u'08.02.2007',
 'day': 8,
 'diff': 0.0,
 'month': 2,
 'mytime': 148.99,
 'rang': 1,
 'ranks': [{'Name': u'ALBRECHT Daniel',
   'Rank': u'1',
   'Time': u'2:28.99',
   'diff': 0.0,
   'race': {u'Bib': u'',
    u'Diff.': u'&nbsp',
    u'FIS Code': u'510993',
    u'FIS Points': u'0.00',
    u'Name': u'ALBRECHT Daniel',
    u'Nation': u'SUI',
    u'Rank': u'1',
    u'Run 1': u'',
    u'Run 2': u'',
    u'Skis': u'?????',
    u'Total Time': u'2:28.99',
    u'Year': u'1983',
    'date': u'08.02.2007',
    'day': 8,
    'diff': 0.0,
    'month': 2,
    'mytime': 148.99,
    'rang': 1,
    u'source_url': u'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=49075',
    u'subevent_': u'',
    u'subevent_Category': u'WSC',
    u's

In [53]:
races_with_time = set([x['raceurl'] for x in all_ergebnisse if x['rang'].strip() and x['time'].strip()])
races_witout_time = set([x['raceurl'] for x in all_ergebnisse if x['rang'].strip() and not x['time'].strip()])

KeyError: 'rang'

In [38]:
len(races_with_time & races_witout_time)

NameError: name 'races_with_time' is not defined

In [ ]:
auswertungen = {
 '1': lambda x: x['Rank']=='1',
 '1 at': lambda x: x['Rank']=='1' and x['Nation']=='AUT',
 '2': lambda x: x['Rank']=='2',
 '2 at': lambda x: x['Rank']=='2' and x['Nation']=='AUT',
 '3': lambda x: x['Rank']=='3',
 '3 at': lambda x: x['Rank']=='3' and x['Nation']=='AUT',
}

all_by_disziplin = {k: list(v) for k,v in sortgroup(all_ergebnisse,
                                                    lambda x: 
                                                    '%s (%s)' % (x['race']['disziplin'],x['race']['gender'],)
                                                   )}

auswertungen_by_disziplin = {
    k: {aname: len(list(x for x in v if aprop(x)))
        for aname, aprop
        in auswertungen.iteritems()
       }
    for k,v in all_by_disziplin.iteritems()
}

In [ ]:
with open('allraces.csv','w') as f:
    cw = unicodecsv.writer(f)
    cw.writerow(['Kategorie','Platz', 'Wert'])
    for k,v in auswertungen_by_disziplin.iteritems():
        for k2, v2 in v.iteritems():
            cw.writerow([k,k2,v2])

In [ ]:
all_ergebnisse[0]